In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

## Load PDF

In [2]:
file_path = "../example_data/PPPR 2024 - Overview.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

66


## Split

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

120

## Embed

In [4]:
embeddings = HuggingFaceEmbeddings(
    model_name="jinaai/jina-embeddings-v3",
    model_kwargs={
        "device":'cuda',
        "trust_remote_code": True
    })

flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn i

In [5]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Vectors length: {len(vector_1)}\n")
print(vector_1[:10])

Vectors length: 1024

[0.0712890625, -0.154296875, -0.035888671875, -0.032470703125, -0.10400390625, -0.09228515625, -0.01904296875, 0.03125, -0.09130859375, -0.00628662109375]


## Vector stores

In [6]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

### Add the doc

In [7]:
ids = vector_store.add_documents(documents=all_splits)

### Try to query

In [8]:
results = vector_store.similarity_search_with_score("Timur Tengah")  # "Middle East" in Bahasa Indonesia

doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.48520073651107326

page_content='country emits. A few small countries with very high per capita emissions (bahrain, Guyana, iceland, Kuwait, oman, 
Palau, Qatar, trinidad and tobago, u nited Arab e mirates) and countries with very low per capita emissions (Central 
African republic, vanuatu) are omitted for visual purposes. t he horizontal axis uses a logarithmic scale.' metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-10-04T09:08:57+05:30', 'author': 'The World Bank Group', 'moddate': '2024-10-04T09:18:42+05:30', 'title': 'The Poverty, Prosperity, and Planet Report 2024; Overview', 'trapped': '/False', 'source': '../example_data/PPPR 2024 - Overview.pdf', 'total_pages': 66, 'page': 46, 'page_label': '31', 'start_index': 1873}


### Use retrievers

In [9]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "Timur Tengah",
        "Asia Selatan",  # South Asia
    ],
)

[[Document(id='ac0dfac9-f138-43e3-aa83-2d1a5e6b9b9a', metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-10-04T09:08:57+05:30', 'author': 'The World Bank Group', 'moddate': '2024-10-04T09:18:42+05:30', 'title': 'The Poverty, Prosperity, and Planet Report 2024; Overview', 'trapped': '/False', 'source': '../example_data/PPPR 2024 - Overview.pdf', 'total_pages': 66, 'page': 46, 'page_label': '31', 'start_index': 1873}, page_content='country emits. A few small countries with very high per capita emissions (bahrain, Guyana, iceland, Kuwait, oman, \nPalau, Qatar, trinidad and tobago, u nited Arab e mirates) and countries with very low per capita emissions (Central \nAfrican republic, vanuatu) are omitted for visual purposes. t he horizontal axis uses a logarithmic scale.')],
 [Document(id='0865dd65-33a6-4852-a69b-d9a2d8bfa869', metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creati

### Dump

In [10]:
vector_store.dump("vector_store.dump")